In [154]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

%matplotlib notebook

In [7]:
def createCircleMask(height, width, rad):
    # rad: ratio from half of height
    circle = np.zeros((height,width))
    skip_circle = cv2.circle(
        circle,(int(width/2),int(height/2)),
        int(height*.5*rad),1,thickness=-1)
    return circle

In [2]:

## 读取图像，解决imread不能读取中文路径的问题
def cv_imread(filePath):
  try:
    cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    ## imdecode读取的是rgb，如果后续需要opencv处理的话，需要转换成bgr，转换后图片颜色会变化
    ##cv_img=cv2.cvtColor(cv_img,cv2.COLOR_RGB2BGR)
    return cv_img
  except:
    traceback.print_exc()
    return None

In [342]:
MAX_FEATURES = 1000
GOOD_MATCH_PERCENT = .9 #0.15

def alignImagesPerspective(im1, im2, mask):
    # Convert images to grayscale
    #im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    #im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    im1Gray = im1
    im2Gray = im2

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, mask)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, mask)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    #matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(descriptors1, descriptors2, None)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imshow("matches.jpg", imMatches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    points1 = np.expand_dims(points1, 0)
    points2 = np.expand_dims(points2, 0)

    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC, 10)
    assert h is not None
    
    print ('alignImagesPerspective')
    print (h)

    # Use homography
    height, width = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h

# !!!!!!!!!!!!!
def alignImagesPerspectiveMulti(im1, im2, mask):
    h = np.identity(3)
    imcur = im1
    sz = im1.shape
    maskcur = mask
    for i in range(15):
        _, h1 = alignImagesPerspective(imcur, im2, mask)
        if np.all(np.abs(h1 - np.identity(3)) < .05):
            break
        h = h1 @ h        
        imcur = cv2.warpAffine(im1,h[:2,:],(sz[1],sz[0]))
        maskcur = cv2.warpAffine(mask,h[:2,:],(sz[1],sz[0]))
        
        cv2.imshow('mask', maskcur)
        cv2.imshow('out', np.hstack((im2,im1,imcur,np.abs(imcur-im2)))/255.)
        cv2.waitKey(10)
    return imcur, h

In [123]:
def alignImagesAffine(im1, im2, mask):
    im1 = im1 * mask
    im2 = im2 * mask
    h = cv2.estimateRigidTransform(im2, im1, False)
    assert h is not None
    h = np.vstack((h, [[0,0,1]]))

    print ('alignImagesAffine:')
    print (h)

    # Use homography
    height, width = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h

In [490]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
RANSAC_TH = 5
def getFlow(im_transformed1, im1):
    '''
    output: 
        flow: size [height, width, 2] 
        flow[:,:,0] is x offset
        flow[:,:,1] is y offset
    '''
    # https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html
    flow = cv2.calcOpticalFlowFarneback(im_transformed1, im1 ,None, 
                                        pyr_scale=0.5, 
                                        levels=7, 
                                        winsize=15,  # 30 good
                                        iterations=20, 
                                        poly_n=5, 
                                        poly_sigma=1.1, flags=0)
    #                                   None, 0.5, 5, 15, 3, 5, 1.5, 0)
    return flow
def getH(flow, mask):
    sz = mask.shape
    flow[...,0] *= mask
    flow[...,1] *= mask
    # Image registration 
    # Ref: CS4243 Computer Vision and Pattern Recognition, NUS - Leow Wee Kheng
    select = np.where((np.abs(flow[...,0]) + np.abs(flow[...,1])).flatten()>1)

    i, j = np.meshgrid(range(sz[0]), range(sz[1]), indexing='ij')
    i1 = i+flow[...,1]
    j1 = j+flow[...,0]
    i = i.flatten()[select]
    j = j.flatten()[select]
    i1 = i1.flatten()[select]
    j1 = j1.flatten()[select]
    M = np.vstack([j,i, np.ones_like(j)]).T
    print (M.shape)
    bj = j1
    bi = i1
    aj = np.linalg.inv(M.T @ M) @ M.T @ bj
    ai = np.linalg.inv(M.T @ M) @ M.T @ bi
    H = np.vstack([aj.T, ai.T, [0,0,1]])
    print ('off=\n', H)
    return H

    #coords = np.vstack([xs, ys]).T
    #coords1 = np.vstack([xs1, ys1]).T
    #M = cv2.getAffineTransform(coords, coords1)
    #print (M)

def getH1(flow, mask):
    sz = mask.shape
    flow[...,0] *= mask
    flow[...,1] *= mask
    # Image registration 
    # Ref: CS4243 Computer Vision and Pattern Recognition, NUS - Leow Wee Kheng
    select = np.where((np.abs(flow[...,0]) + np.abs(flow[...,1])).flatten()>0.001)
    if (len(select[0])<3):
        return np.identity(3)
    i, j = np.meshgrid(range(sz[0]), range(sz[1]), indexing='ij')
    i1 = i+flow[...,1]
    j1 = j+flow[...,0]
    i = i.flatten()[select]
    j = j.flatten()[select]
    i1 = i1.flatten()[select]
    j1 = j1.flatten()[select]
    H, mask = cv2.findHomography(np.vstack([j,i]).T, np.vstack([j1,i1]).T,
                                 cv2.RANSAC, RANSAC_TH)
    print (H)
    return H
    
def showFlow(im_ori, im_transformed, flow, mask):
    flow[...,0] *= mask
    flow[...,1] *= mask
    
    sz = im_ori.shape
    im_out = np.zeros(sz, dtype=np.float32)
    print (im_out.shape)
    for i in range(sz[0]):
        for j in range(sz[1]):
            i1 = int(i+flow[i,j,1])
            j1 = int(j+flow[i,j,0])
            if i1>=0 and j1>=0 and i1<sz[0] and j1<sz[1]:
                im_out[i1][j1] = im_transformed[i,j]
    #plt.figure()
    #plt.imshow(np.hstack((im_ori, im_out)),  cmap='gray')    
    cv2.imshow('flow map', np.hstack((im_ori, im_out))/255.)
    
def showFlow1(im_ori, im_transformed, flow, mask):
    flow[...,0] *= mask
    flow[...,1] *= mask
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv = np.zeros(im_ori.shape+(3,))
    hsv[...,0] = ang*180/np.pi
    hsv[...,1] = 255
    hsv[...,2] = mag/50 #cv2.normalize(mag,None,0,1,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv.astype(np.float32),cv2.COLOR_HSV2RGB)
    cv2.imshow('flow', bgr)
    
def alignImagesOpticalFlow(im1, im2, mask):
    mask = np.float32(mask)
    im1 = np.float32(im1)
    im2 = np.float32(im2)
    imcur = im1
    sz = im1.shape
    
    H0 = np.identity(3)
    rads = 16
    
    best_h = H0
    best_err = 9999
    
    for r in range(0,rads):
        # rotate
        T_shift_neg = np.array([
            [1, 0, -sz[1]*.5],
            [0, 1, -sz[0]*.5],
            [0, 0, 1]])
        T_shift = np.array([
            [1, 0, sz[1]*.5],
            [0, 1, sz[0]*.5],
            [0, 0, 1]])
        th = np.pi/(rads/2)
        T_rotate = np.array([
            [np.cos(th), -np.sin(th), 0],
            [np.sin(th), np.cos(th), 0],
            [0, 0, 1]])
        H0 = T_shift @ T_rotate @ T_shift_neg @ H0
        H1 = H0
        #H1 = np.array([[1,0,10], [0,1,0], [0,0,1]], dtype=np.float)
        imcur = cv2.warpAffine(im1,H1[:2,:],(sz[1],sz[0]))
        mask = cv2.warpAffine(mask,H1[:2,:],(sz[1],sz[0]))
        cv2.imshow('mask', mask)

        mask1=mask

        for i in range(5):
            flow = getFlow(imcur, im2)

            showFlow1(im1, imcur, flow, mask1)        

            H = getH1(flow, mask1)
            
            H1 = H @ H1
            
            det = np.linalg.det(H1)
            print ("H1 Det:", det)
            if det>2 or det<.5:
                break
                
            imcur = cv2.warpAffine(im1,H1[:2,:],(sz[1],sz[0]))
            mask1 = cv2.warpAffine(mask,H1[:2,:],(sz[1],sz[0]))
            cv2.imshow('mask', mask1)
            
            err = np.mean(abs(imcur-im2)[mask.astype('bool')])
            if err < best_err:
                print ('best_err = ', best_err)
                best_err = err
                best_h = H1

            print (np.max(imcur), np.max(im2))
            print (np.max(np.abs(imcur-im2)))
            #plt.figure()
            print (im1.shape)
            #plt.imshow(np.hstack((im2,im1,imcur,np.abs(imcur-im2))) ,  cmap='gray')
            #plt.pause(.01)
            cv2.imshow('out', np.hstack((im2,im1,imcur*mask1,np.abs(imcur-im2)))/255.)
            k = cv2.waitKey(1)
            if k==32:
                break
    return best_h

In [ ]:

#refFilename = "D:/Images/A70/Image_60057B1/鏡筒_A_IPI_4_20180510_154021893.png"
#imFilename = "D:/Images/A70/Image_60057B1/鏡筒_A_IPI_4_20180510_154029071.png"
#mask_rad=.4
#refFilename = "D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040155115.png"
#imFilename = "D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png"
#mask_rad=.2

refFilename = "D:\Images\A70\Image_60057B1/鏡筒_A_IPI_4_20180510_154021893.png"
path = "D:\Images\A70\Image_60057B1/"
mask_rad=.4
#refFilename = "D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040155115.png"
#path = "D:\Images\A70\Image_8419A5/"
#mask_rad=.2

mask = 1-createCircleMask(im.shape[0], im.shape[1], mask_rad).astype('uint8')
mask *= createCircleMask(im.shape[0], im.shape[1], .8).astype('uint8')

step = 8

# Read reference image
print("Reading reference image : ", refFilename)
imReference = cv_imread(refFilename)
imReference = imReference[::step, ::step]
cv2.imshow('ref', imReference)

# Create array of array of images.
imagePaths = []
i = 0        
k = 0
for root, subdirs, files in os.walk(path):
    print("Reading images from " + root + " ... ", flush=True)      
    for filename in files:
        fileExt = os.path.splitext(filename)[1]
        if fileExt not in [".jpg", ".jpeg", ".png"]:
            continue

        imagePath = os.path.join(root, filename)
            
        # Read image to be aligned
        print("Reading image to align : ", imFilename);  
        im = cv_imread(imagePath)
        im = im[::step, ::step]
        cv2.imshow('im', im)

        if im is None :
            print("Error:{} not read properly".format(imagePath))
            continue
        
        #Reference *= mask
        #im *= mask
        
        print("Aligning images ...")
        # Registered image will be resotred in imReg. 
        # The estimated homography will be stored in h. 
        
        #imReg, h = alignImagesPerspectiveMulti(im, imReference, mask)
        #imReg, h = alignImagesAffine(imReg, imReference, mask)
        h = alignImagesOpticalFlow(im, imReference, mask)
        sz = imReference.shape
        imReg = cv2.warpAffine(im,h[:2,:],(sz[1],sz[0]))        
        
        
        diff = np.abs(imReg.astype('float')-imReference.astype('float')).astype('uint8')
        cv2.imshow('out', np.hstack([imReference, im, imReg, diff ]))

        # Write aligned image to disk. 
        #outFilename = "aligned.jpg"
        #print("Saving aligned image : ", outFilename); 
        #cv2.imwrite(outFilename, imReg)

        # Print estimated homography
        print("Estimated homography : \n",  h)
        print ('Press key to cont.')
        k = cv2.waitKey(0)

        if k==27:
            break
    if k==27:
        break
print ('Done')

Reading reference image :  D:\Images\A70\Image_60057B1/鏡筒_A_IPI_4_20180510_154021893.png
Reading images from D:\Images\A70\Image_60057B1/ ... 
Reading image to align :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png
Aligning images ...
[[ 1.07267117e+00  1.70644631e-01 -1.29639477e+01]
 [-8.96113856e-02  1.12376658e+00  6.81946416e+00]
 [ 1.07817969e-04  7.46492390e-04  1.00000000e+00]]
H1 Det: 1.2178265114611946
best_err =  9999
255.0 255.0
255.0
(150, 200)
[[ 1.11656964e+00  1.50778517e-01 -1.65557287e+01]
 [-9.53261455e-02  1.04256335e+00  1.09131211e+01]
 [ 8.16502170e-04  4.14526965e-04  1.00000000e+00]]
H1 Det: 1.448613745493979
255.0 255.0
255.0
(150, 200)
[[ 6.41335356e-01  7.60292079e-02  4.44292809e+00]
 [-2.59920401e-01  7.55283861e-01  2.36475976e+01]
 [-2.42330333e-03  2.14474640e-04  1.00000000e+00]]
H1 Det: 0.7307188895906864
255.0 255.0
255.0
(150, 200)
[[ 1.19498607e+00  1.84838520e-02 -5.65819205e+00]
 [ 9.26923374e-02  1.06995771e+00 -3.01221280e+00]
 [

[[ 8.94795506e-01  3.43287577e-02  2.43795975e+00]
 [-8.96808121e-02  1.02026498e+00  5.19807425e+00]
 [-5.99223915e-04  3.31925522e-04  1.00000000e+00]]
H1 Det: 0.9505057352656321
255.0 255.0
255.0
(150, 200)
[[ 1.26418829e+00  2.19492228e-01 -2.66749541e+01]
 [-3.62050370e-02  1.03847463e+00  8.50860751e+00]
 [ 2.61976736e-04  9.19843761e-04  1.00000000e+00]]
H1 Det: 1.254206209025451
255.0 255.0
255.0
(150, 200)
[[ 8.20370517e-01  7.78416205e-02  3.26343287e-01]
 [-1.39539618e-01  8.90441608e-01  1.70380181e+01]
 [-4.15179920e-04  1.97167630e-05  1.00000000e+00]]
H1 Det: 0.928924757443818
255.0 255.0
255.0
(150, 200)
[[ 1.02852320e+00  6.34684626e-03 -1.86838473e+00]
 [-5.85543403e-02  9.84452147e-01  6.30332225e+00]
 [-7.32612654e-05  2.90221439e-05  1.00000000e+00]]
H1 Det: 0.9406114172048693
255.0 255.0
255.0
(150, 200)
[[ 9.62609082e-01 -1.16854924e-02  1.57080855e+00]
 [-1.57857041e-02  9.77406869e-01  1.50735725e+00]
 [-1.83453751e-04 -4.52631388e-05  1.00000000e+00]]
H1 Det: 

255.0 255.0
255.0
(150, 200)
[[ 1.44240452e+00 -2.17249450e-01 -1.86947772e+01]
 [ 2.64852720e-01  9.90371979e-01 -1.95786088e+01]
 [ 1.64808110e-03 -1.81101385e-03  1.00000000e+00]]
H1 Det: 1.2328072607542486
255.0 255.0
255.0
(150, 200)
[[ 7.25054672e-01 -4.82082378e-02  2.82871452e+01]
 [-3.31154185e-02  1.04682112e+00 -2.66223329e+00]
 [ 1.79312787e-05 -2.58080756e-04  1.00000000e+00]]
H1 Det: 0.9327678990364926
255.0 255.0
255.0
(150, 200)
[[ 9.53551323e-01 -1.10221992e-02  2.65673942e+00]
 [ 6.08012128e-03  9.74569845e-01  1.30934225e-01]
 [-2.11915070e-04 -2.09224395e-04  1.00000000e+00]]
H1 Det: 0.8674192210448879
255.0 255.0
255.0
(150, 200)
[[ 1.07055752e+00  8.08075683e-02 -9.40099966e+00]
 [-5.36048183e-02  1.04801810e+00  5.29392592e+00]
 [ 2.62052535e-04  1.42830992e-04  1.00000000e+00]]
H1 Det: 1.128251776606258
255.0 255.0
255.0
(150, 200)
[[ 9.65713839e-01  3.19659954e-02 -6.12394022e-01]
 [-5.16702611e-02  9.72917058e-01  5.81969193e+00]
 [ 2.48412381e-04 -1.10174441e

 [-4.69115565e-05  1.04916587e-04  1.00000000e+00]]
H1 Det: 1.0152207299440266
255.0 255.0
255.0
(150, 200)
[[ 1.01025921e+00 -3.31221775e-02 -1.01109004e+00]
 [-4.16021364e-04  9.26438739e-01  2.68891327e+00]
 [ 1.25229193e-04 -4.64723248e-04  1.00000000e+00]]
H1 Det: 0.9372978652491168
255.0 255.0
255.0
(150, 200)
[[ 1.04769431e+00 -1.08998363e-01 -1.80110219e-01]
 [ 1.83279106e-02  1.00349417e+00 -1.91352252e+00]
 [ 3.17628683e-04 -8.61986110e-04  1.00000000e+00]]
H1 Det: 0.9858041952871073
255.0 255.0
255.0
(150, 200)
[[ 1.03776413e+00  1.08497540e-01 -2.85397271e+00]
 [ 5.94232566e-02  1.08591727e+00 -7.75654472e+00]
 [ 4.52752100e-04 -1.65033733e-04  1.00000000e+00]]
H1 Det: 1.1042982587139827
255.0 255.0
255.0
(150, 200)
[[ 1.01744541e+00  9.41909597e-03 -2.17510277e+00]
 [-4.76836434e-03  9.91369103e-01  1.31559896e+00]
 [ 2.20744886e-04 -4.19130725e-06  1.00000000e+00]]
H1 Det: 1.1144502447105085
255.0 255.0
255.0
(150, 200)
[[ 8.99169483e-01 -3.75322043e-02  7.49182700e+00]
 

[[ 9.56966321e-01 -1.02177457e-02  1.80335062e+00]
 [-1.89350800e-02  9.58697561e-01  2.37417184e+00]
 [-3.46065483e-04 -2.11953242e-04  1.00000000e+00]]
H1 Det: 0.8586152975936489
255.0 255.0
255.0
(150, 200)
[[ 1.03869485e+00  2.41031662e-02 -3.04965912e+00]
 [ 1.45820336e-02  1.03565083e+00 -2.16245003e+00]
 [-2.31926982e-04 -1.04755907e-04  1.00000000e+00]]
H1 Det: 0.9225157258092812
255.0 255.0
255.0
(150, 200)
[[ 1.00458978e+00  3.91552541e-02 -3.49041904e+00]
 [-4.09470455e-02  9.91374442e-01  4.95679305e+00]
 [-1.22165098e-05  8.84326023e-06  1.00000000e+00]]
H1 Det: 0.997440506402711
255.0 255.0
255.0
(150, 200)
[[ 9.98919616e-01  4.56914922e-02 -3.11496270e+00]
 [-5.13752484e-02  1.00880981e+00  4.96519417e+00]
 [-2.27177254e-05  6.34783888e-05  1.00000000e+00]]
H1 Det: 1.0071018042129407
255.0 255.0
255.0
(150, 200)
[[ 9.96143123e-01  8.01149365e-02 -4.75881407e+00]
 [-5.26243073e-02  1.02583588e+00  4.52237286e+00]
 [-1.00386604e-04  2.82456618e-04  1.00000000e+00]]
H1 Det:

...

255.0 255.0
255.0
(150, 200)
[[ 9.94660717e-01 -1.64605618e-02  8.28132478e-01]
 [ 1.17581995e-02  9.55949327e-01  6.15746842e-01]
 [-2.78329500e-04  2.40863847e-04  1.00000000e+00]]
H1 Det: 0.9613490761667511
255.0 255.0
255.0
(150, 200)
[[ 9.49404422e-01 -8.89849983e-04  7.02135137e-01]
 [-3.24281583e-02  9.51295447e-01  3.90992285e+00]
 [-2.41311877e-04 -1.41096292e-04  1.00000000e+00]]
H1 Det: 0.903824244902456
255.0 255.0
255.0
(150, 200)
[[ 1.10914838e+00 -2.85328220e-02 -6.95652360e+00]
 [ 4.41981098e-02  9.76698055e-01 -1.91454211e+00]
 [ 2.49123075e-04 -5.76001044e-04  1.00000000e+00]]
H1 Det: 0.980852096411716
255.0 255.0
255.0
(150, 200)
[[ 1.06566475e+00  3.31709645e-02 -5.57659482e+00]
 [ 4.15034726e-03  1.06833300e+00 -2.23815504e+00]
 [ 1.55152833e-04  3.38787512e-05  1.00000000e+00]]
H1 Det: 1.1175240222907712
255.0 255.0
255.0
(150, 200)
[[ 1.00902672e+00 -2.74409126e-03  2.47353702e+00]
 [ 1.60414925e-02  1.03997757e+00 -2.29180057e+00]
 [ 3.40410902e-04  6.24321674e-

255.0 255.0
255.0
(150, 200)
[[ 1.04134529e+00  3.88765870e-02 -3.73986417e+00]
 [-1.05546165e-02  1.06240287e+00 -5.00928879e-01]
 [ 3.22861349e-04  5.60527509e-04  1.00000000e+00]]
H1 Det: 1.2580427167501484
255.0 255.0
255.0
(150, 200)
[[ 1.28737585e+00 -3.07192551e-02 -1.39388213e+01]
 [ 1.02914728e-01  1.15364839e+00 -1.02342200e+01]
 [ 1.02911517e-03  3.19660378e-04  1.00000000e+00]]
H1 Det: 1.5089658277411935
255.0 255.0
255.0
(150, 200)
[[ 7.30641479e-01 -1.22168625e-01  1.75340255e+01]
 [ 5.52302119e-02  8.27715334e-01 -6.95092616e+00]
 [-5.49026807e-04 -1.60721232e-03  1.00000000e+00]]
H1 Det: 0.9194031815486602
255.0 255.0
255.0
(150, 200)
[[ 1.02448122e+00 -1.77677705e-01  7.69266068e+00]
 [ 6.37685650e-02  8.62400476e-01  3.32657378e-01]
 [ 3.85425618e-05 -1.25308310e-03  1.00000000e+00]]
H1 Det: 0.8223121128302489
255.0 255.0
255.0
(150, 200)
[[ 1.36546709e+00  1.01231959e-01 -1.44513034e+01]
 [ 1.49808734e-01  1.39979946e+00 -2.05907598e+01]
 [ 1.71916215e-03 -1.57064262

[[ 7.54315157e-01 -2.76560390e-02  1.15478004e+01]
 [-1.39916742e-01  8.28604930e-01  1.54340257e+01]
 [-7.33988103e-04 -1.57174145e-05  1.00000000e+00]]
H1 Det: 0.8268782678790498
255.0 255.0
255.0
(150, 200)
[[ 1.01962896e+00 -1.07571898e-02  4.39798150e-01]
 [ 2.10352962e-02  1.02609213e+00 -2.65329491e+00]
 [ 8.50916926e-05  1.01025541e-04  1.00000000e+00]]
H1 Det: 1.0466978119732817
255.0 255.0
255.0
(150, 200)
[[ 1.00327980e+00 -4.37785079e-02  1.81444831e+00]
 [ 2.07981099e-02  9.90943270e-01 -1.58074724e+00]
 [-2.77062326e-05  1.26379729e-04  1.00000000e+00]]
H1 Det: 1.0418379707848406
255.0 255.0
255.0
(150, 200)
[[ 1.04707689e+00  6.63362320e-02 -3.99648884e+00]
 [ 1.79668041e-02  1.13644388e+00 -4.79869657e+00]
 [-7.72370055e-05  1.05406109e-03  1.00000000e+00]]
H1 Det: 1.2435863697293448
255.0 255.0
255.0
(150, 200)
[[ 1.00336010e+00 -2.58028971e-02  4.08794774e-01]
 [-1.69271112e-02  9.50271780e-01  3.73102068e+00]
 [-3.42272548e-06  6.33998414e-04  1.00000000e+00]]
H1 Det

255.0 255.0
255.0
(150, 200)
[[ 1.15346847e+00 -7.58419347e-03 -5.86793112e+00]
 [ 8.09673050e-02  1.07989652e+00 -7.46578721e+00]
 [ 3.20033872e-05  2.08635138e-04  1.00000000e+00]]
H1 Det: 1.0767841971849488
255.0 255.0
255.0
(150, 200)
[[ 9.26591493e-01  2.42271008e-02  3.98926896e+00]
 [-2.92531972e-02  1.03814788e+00  2.41884008e-01]
 [-2.97055650e-04  4.79607866e-04  1.00000000e+00]]
H1 Det: 1.0377106601379038
255.0 255.0
255.0
(150, 200)
[[ 1.12107187e+00  3.10452941e-02 -6.80665407e+00]
 [ 2.62466720e-02  1.05322977e+00 -2.49647068e+00]
 [ 3.99870042e-04  3.36372949e-04  1.00000000e+00]]
H1 Det: 1.2282845794899313
255.0 255.0
255.0
(150, 200)
[[ 9.71533502e-01 -3.52281461e-02  1.35312442e+00]
 [-1.14360157e-02  9.42174431e-01  2.77891447e+00]
 [-1.16249378e-04 -2.56443075e-04  1.00000000e+00]]
H1 Det: 0.9158070552074513
255.0 255.0
255.0
(150, 200)
[[ 1.02993518e+00  2.74742020e-02 -2.45113895e+00]
 [ 5.30350752e-03  1.08247341e+00 -2.91510770e+00]
 [-1.98091424e-04  3.31948319

255.0 255.0
255.0
(150, 200)
[[ 9.39285224e-01 -2.24082557e-02  5.12043029e+00]
 [-7.28072551e-03  9.92539856e-01  1.22018113e-01]
 [-6.94535697e-05  4.85393379e-04  1.00000000e+00]]
H1 Det: 1.0990494286019317
255.0 255.0
255.0
(150, 200)
[[ 9.94227964e-01 -7.16184713e-03  4.94136450e-01]
 [ 3.67403643e-03  9.99641440e-01 -6.23995734e-01]
 [-6.53397599e-05  4.38951756e-04  1.00000000e+00]]
H1 Det: 1.0926781179534957
255.0 255.0
255.0
(150, 200)
[[ 1.14332894e+00  3.28043218e-02 -7.70225053e+00]
 [ 4.13493587e-02  1.09387345e+00 -4.95807198e+00]
 [ 6.53165134e-04  5.52807139e-04  1.00000000e+00]]
H1 Det: 1.3742123553329275
255.0 255.0
255.0
(150, 200)
[[ 9.30670860e-01 -4.22339306e-02  5.00238245e+00]
 [-2.77530468e-02  9.21762868e-01  4.36208296e+00]
 [-2.55667862e-04 -2.90821653e-04  1.00000000e+00]]
H1 Det: 0.8591327230673044
255.0 255.0
255.0
(150, 200)
[[ 8.83536156e-01 -2.26974644e-02  5.29665070e+00]
 [-1.86854695e-03  9.42665214e-01 -6.72269561e-01]
 [-9.49624939e-04 -2.36871273

In [452]:
cv2.__version__
#imReg = cv2.warpAffine(im,h[:2,:],(sz[1],sz[0]))        
h

(array([[128, 126, 128, ...,  86,  88,  90],
        [134, 129, 133, ...,  92,  94,  94],
        [133, 122, 126, ...,  86,  88,  94],
        ...,
        [ 97, 103,  99, ..., 131, 132, 130],
        [ 99,  99, 101, ..., 132, 140, 143],
        [103, 102, 102, ..., 134, 138, 138]], dtype=uint8),
 array([[ 1.00059380e+00, -4.92590202e-04, -1.02109904e-03],
        [ 4.67753625e-05,  1.00104664e+00, -1.20936807e-01],
        [-3.72236765e-05,  5.54675533e-04,  9.69863400e-01]]))